In [19]:
import numpy as np
import pandas as pd

np.seterr(divide='ignore')

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'ignore', 'under': 'ignore'}

In [49]:
def residual_band(C,paths):
    '''Computes the residual bandwidth for all input paths given the residual bandwidth'''
    
    res_band =[]
    for ii in range(len(paths)):
        vec = []
        for tt in paths[ii]:
            vec.append(C[tt])
        res_band.append(np.min(vec))

    return res_band

def intersect(a, b):
    return list(set(a) & set(b))


def max_min_allocation(C,paths, debug_flag=0):
    '''Computes the max-min allocation in a capacitated graph G for all input paths'''
    
    C = np.array(C, dtype=float)
    paths=np.array(paths)
    # check input consistency
    if (np.sum(C<0)>0):
        raise ValueError('Capacity must be nonnegative.')
    
    # Initialization
    max_min_rates = np.zeros(len(paths))

    N = len(C) # n. edges
    n_paths_per_edge = np.zeros(N, dtype=int) # n_paths_per_edge[i] = n. of paths passing through edge i
    for val in paths:
        for tt in val:
            n_paths_per_edge[tt] += 1

    paths_not_bottlenecked = [True for ii in range(len(paths))] #np.ones([1,len(paths)],dtype=bool)
    
    # Max-min algorithm
    it = 0;
    
    if(debug_flag):
        print("iteration " + str(it))
        print("max_min_rates = " + str(max_min_rates))
        print("paths_not_bottlenecked = " + str(paths_not_bottlenecked))
        print("available bandwidth C = " + str(C))
        print("n_paths_per_edge = " + str(n_paths_per_edge) + "\n")
       
    
    while np.sum(paths_not_bottlenecked)>0:
        
        vec = np.array([float("Inf") for ii in range(len(C))])
        bool_vec = (n_paths_per_edge>0)
        vec[bool_vec] = C[bool_vec] / n_paths_per_edge[bool_vec]
        
        bottleneck_flow = np.nanmin(vec) # maximum amount of flow increment for not bottlenecked flows
        bottleneck_links = np.where(vec==bottleneck_flow)[0] # links with no capacity left
        paths_not_bottlenecked_ind=np.where(paths_not_bottlenecked==True)
        max_min_rates[paths_not_bottlenecked_ind] += bottleneck_flow
        
        ii=0
        for val in paths[paths_not_bottlenecked_ind].iteritems():
            C[val] -= bottleneck_flow
            if intersect(val,bottleneck_links):
                paths_not_bottlenecked[ii] = False
                for tt in val:
                    n_paths_per_edge[tt] -= 1
            ii+=1
        # import pdb; pdb.set_trace() # DEBUG
        it += 1
        
        if(debug_flag):
            print("iteration " + str(it))
            print("max_min_rates = " + str(max_min_rates))
            print("paths_not_bottlenecked = " + str(paths_not_bottlenecked))
            print("available bandwidth C = " + str(C))
            print("n_paths_per_edge = " + str(n_paths_per_edge) + "\n")
    
    
    return max_min_rates

In [50]:
C = [5,6,1,3,10] # C[i] is the capacity of link i
paths = [[0,3],[0,2,4],[1,4]] # set of paths, meant as sequence of edges

max_min_rates = max_min_allocation(C,paths,1)
max_min_rates

iteration 0
max_min_rates = [ 0.  0.  0.]
paths_not_bottlenecked = [True, True, True]
available bandwidth C = [  5.   6.   1.   3.  10.]
n_paths_per_edge = [2 1 1 1 2]



AttributeError: 'numpy.ndarray' object has no attribute 'iteritems'

In [10]:
t1 = np.array([0,3,6,7], dtype=float)
t2 = np.array([0,0,9,6], dtype=float)
v = np.array([float("Inf") for ii in range(len(t1))])
bo = (t1!=0) & (t2!=0)
v[bo] = t1[bo] / t2[bo]
v

array([        inf,         inf,  0.66666667,  1.16666667])

In [21]:
for aa bb in enumerate(pd.Series(paths)):
    print aa, bb

SyntaxError: invalid syntax (<ipython-input-21-80d2bcd44e12>, line 1)

In [33]:
max_min_rates = np.zeros(3)
print max_min_rates
print max_min_rates

[ 0.  0.  0.]
[ 0.  0.  0.]


In [35]:
print max_min_rates

[ 1.  0.  1.]


In [34]:
max_min_rates[[0,2]]+=1

In [32]:
max_min_rates[[0,2]]

array([ 1.,  0.])

In [40]:
max_min_rates[np.where(np.array([True,False,True])==True)]

array([ 1.,  1.])